<a href="https://colab.research.google.com/github/aidyai/rss-ai/blob/main/rss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade --quiet  apify-client langchain-openai langchain
!pip install feedparser newspaper3k openai
#!pip install openai==0.28
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document

import feedparser
import requests
from bs4 import BeautifulSoup
import openai
from newspaper import Article

In [ ]:
import os
os.environ["APIFY_API_TOKEN"] = ""
os.environ["OPENAI_API_KEY"] = ""


## ARTICLE EXTRACTION: This code only extracts onearticle but can be modified to work for the latest 10, 50 or 20 articles

In [ ]:
# RSS feed URL
rss_feed_url = 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen'

# Parse the RSS feed
feed = feedparser.parse(rss_feed_url)

# Get the latest just one articles
latest_articles = feed.entries[8:9]   # we just yse one for speed

In [ ]:
print(latest_articles)

[{'title': 'Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers', 'title_detail': {'type': 'text/plain', 'language': None, 'base': 'https://news.google.com/rss/search?q=akwa%20ibom&hl=en-US&gl=US&ceid=US%3Aen', 'value': 'Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers'}, 'links': [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}], 'link': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5', 'id': 'CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA', 'guidislink': False, 'published': 'Sat, 04 May 2024 16:27:45 GMT', 'published_parsed':

In [ ]:
apify = ApifyWrapper()

In [ ]:
title = []
urls_to_crawl = []

for article in latest_articles:

    # extracting the title and article URL
    title = article.title
    article_url = article.link



    # Appending the article title and URL
    #title.append({"title": title})
    urls_to_crawl.append({"url": article_url})

In [ ]:
print(urls_to_crawl)

[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


In [ ]:
#this functions looks into the urls_to_crawl and extracts its text
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": urls_to_crawl},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Extracting the title, article_url and article_text for the latest 10 articles

In [ ]:
print(title)                      #EXTRACTED TITLE
print(urls_to_crawl)              #EXTRACTED URL

article_text = loader.load()
article_text                      #EXTRACTED ARTICLE TEXY

Akpabio To Eno: You have a divine duty to unite Akwa Ibom People - Champion Newspapers
[{'url': 'https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vY2hhbXBpb25uZXdzLmNvbS5uZy9ha3BhYmlvLXRvLWVuby15b3UtaGF2ZS1hLWRpdmluZS1kdXR5LXRvLXVuaXRlLWFrd2EtaWJvbS1wZW9wbGUv0gEA?oc=5'}]


[Document(page_content='President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy th

In [ ]:
import json
# Extracting content from the list
extracted_content = [document.page_content for document in article_text]

In [ ]:
extracted_content

['President of the Senate, Godswill Akpabio has charged the Akwa Ibom State Governor, Pastor Umo Eno, on the need to unite the state across party lines for the continued development and progress of the state and not make his father’s enemies his enemies.\nHe described the Governor as a man of peace and an Apostle and disciple of peace, noting, “Akwa Ibom is a very easy state to rule and govern but also a very difficult place to unite.”\nAkpabio gave the charge while on a courtesy visit to the Governor in his office, in Uyo on Friday.\nA statement by his Special Assistant on Media, Jackson Udom, quoted the Senate President as saying, “it is a major duty bestowed on you by God to unite the whole state. Let your actions speak unity to the the state. As a governor, I united the state by appointing people into key positions without considering where they came from.\n“I want you to unite your people without any political considerations. Even if you inherited a policy that was not in the best

In [ ]:
import openai
# Send the content to OpenAI to get topic and category tags
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=extracted_content,
    max_tokens=50,  # Adjust as needed
    n=1,  # Number of completions to generate
    stop=["\n"],  # Stop completion at the end of the text
    temperature=0.5,  # Adjust as needed
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    logit_bias={},
    logprobs=0,
    echo=True  # Print the input prompt along with the completion
)


# USING REPLICATE API TO CALL POWERFUL LARGE LANGUAGE MODELS

In [ ]:
import replicate

In [ ]:
import os
import replicate
from replicate.client import Client

replicate = Client(api_token="")

## USING MISTRAL 8x7B via Replicate Api

In [ ]:
for event in replicate.stream(
    "mistralai/mixtral-8x7b-instruct-v0.1",
    input={
        "top_k": 50,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSIATAT THAT CAN GENERATE topic tags AND  article category tags",
        "length_penalty": 1,
        "max_new_tokens": 1024,
        "prompt_template": "<s>[INST] {prompt} [/INST] ",
        "presence_penalty": 0
    },
):
    print(str(event), end="")

ReplicateError: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

## USING METAS LLAMA 3 Via Replicate Api

In [ ]:
for event in replicate.stream(
    "meta/meta-llama-3-70b-instruct",
    input={
        "top_k": 0,
        "top_p": 0.9,
        "prompt": f"Generate 5 'TOPIC TAGS' AND 'CATEGPRY TAGS' for the given article\ {extracted_content}",
        "max_tokens": 650,
        "min_tokens": 0,
        "temperature": 0.6,
        "system_prompt": "YOU ARE AN AI ASSITANT THAT CAN GENERATE topic tags AND article category tags",
        "length_penalty": 1,
        "stop_sequences": "<|end_of_text|>,<|eot_id|>",
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "presence_penalty": 1.15,
        "log_performance_metrics": False
    },
):
    print(str(event), end="")

Here are five topic tags and five category tags for the given article:

**Topic Tags:**

1. Oil Production
2. Nigeria Energy Sector
3. NNPC E&P Ltd
4. Natural Oilfield Services Ltd
5. Oil Mining Lease (OML) 13

**Category Tags:**

1. Business
2. Energy
3. Nigeria News
4. Oil and Gas
5. Economy

## RECIEVES NOTIFICATION

In [2]:
def ask_user():
    user_input = input("Please enter something: ")
    print(f"You entered: {user_input}")


In [ ]:
# Call the function
ask_user()
